# Notebook to extract climate data and load Uber H3 polygons.

In [175]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import h3
import folium
from folium.plugins import HeatMap
from typing import List, Union, Optional
from pathlib import Path
from datetime import date, datetime
import requests
import os

##  Data

In [ ]:
#Jaguar's Data'
df = pd.read_csv('jaguar_movement_data.csv')

In [177]:
df.columns

Index(['Event_ID', 'timestamp', 'location.long', 'location.lat',
       'individual.taxon.canonical.name', 'tag.local.identifier',
       'individual.local.identifier (ID)', 'study.name', 'country'],
      dtype='object')

In [178]:
df.rename(columns={
    'location.long': 'location_long', 
    'location.lat': 'location_lat',
    'individual.taxon.canonical.name': 'canonical_taxonomy',
    'individual.local.identifier (ID)': 'ID_individual_identifier',
    'study.name': 'study_name',
    'tag.local.identifier': 'tag_local_identifier',
  
    }, inplace=True)

### Aplying H3 Polygons

In [ ]:
#import h3

#Polygons H· (8[0.72Km^2] y 6[36Km^2] )
df['polygon_8'] = df.apply(lambda row: h3.latlng_to_cell(row['location_lat'], row['location_long'], 8), axis=1)
df['polygon_6'] = df.apply(lambda row: h3.latlng_to_cell(row['location_lat'], row['location_long'], 6), axis=1)

In [182]:
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%m/%d/%y %H:%M")

In [183]:
df['timestamp_D'] = df['timestamp'].dt.floor("D")

### Wheather Data Extraction Function

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime

def get_weather_data(lat: float, lon: float, date: datetime):
    """
    Queries the NASA POWER API to fetch daily weather data.
    """
    url = "https://power.larc.nasa.gov/api/temporal/daily/point?"
    params = {
        "parameters": "T2M,RH2M,WS10M,PS,PRECTOTCORR",
        "community": "RE",
        "longitude": lon,
        "latitude": lat,
        "start": date.strftime("%Y%m%d"),
        "end": date.strftime("%Y%m%d"),
        "format": "JSON"
    }
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            return data["properties"]["parameter"]
        else:
            print(f"Error in the query: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception in the query: {e}")
        return None

def fetch_weather_in_batches(df, batch_size=100, output_file='weather_data.csv'):
    """
    Performs queries in batches and progressively saves the results.
    """
    weather_data_list = []
    total_rows = len(df)
    
    for i in range(0, total_rows, batch_size):
        batch = df.iloc[i:i+batch_size]
        temp_results = []
        
        for _, row in batch.iterrows():
            weather_data = get_weather_data(row['location_lat'], row['location_long'], row['timestamp_D'])
            if weather_data:
                weather_data['location_lat'] = row['location_lat']
                weather_data['location_long'] = row['location_long']
                weather_data['timestamp_D'] = row['timestamp_D']
                temp_results.append(weather_data)
            time.sleep(1)  # Prevents overloading the API
        
        if temp_results:
            temp_df = pd.DataFrame(temp_results)
            temp_df.to_csv(output_file, mode='a', index=False, header=not bool(i))  # Saves data progressively
            print(f"Saved block {i} - {i+batch_size} to {output_file}")
        
    print("Process completed")

# Assume df_4 is already loaded and contains the columns 'latitude', 'longitude', and 'date'
fetch_weather_in_batches(df)